# Data Science en Entreprise

## API pôle emploi

Autorisation d’accès à l’api des offres d’emplois

Compte pole emploi :    
username : mohammed.didi@outlook.com    
password : Projet_Data1



In [1]:
import requests


def obtenir_token(client_id, client_secret):
    """
    Cette fonction obtient un token d'accès pour l'API de Pôle Emploi.
    """
    url_token = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire'
    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': 'api_offresdemploiv2 o2dsoffre'
    }
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}

    response = requests.post(url_token, data=auth_data, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Erreur lors de la requête : {response.text}")
    else:
        data = response.json()
        return data['access_token']
    
# Utilisation de la fonction
client_id = 'PAR_projetdata_29c785675807663802eabdf57858fc4e2ecd4eba4de8d05eee293bfdcacefae5'
client_secret = '8a48c4726bb2d33cafb7a636703a3b0ad70e4098b525063d2a99387a95a76b70'


In [24]:
import requests
from datetime import datetime
import pandas as pd
import json


def rechercher_offres(token, mot_cle, mois, annee, nb_offres=10):
    """
    Recherche les offres d'emploi dans le domaine des RH pour un mois et une année spécifiques.
    """
    url_offres = 'https://api.emploi-store.fr/partenaire/offresdemploi/v2/offres/search'
    headers = {'Authorization': f'Bearer {token}'}

    # Calculer les dates de début et de fin du mois de novembre
    debut_mois = f"{annee}-{mois:02d}-01T00:00:00Z"
    fin_mois = f"{annee}-{mois:02d}-30T00:00:00Z"  # Novembre a 30 jours

    params = {
        'motsCles': mot_cle,  # Mots-clés pour le domaine des Ressources Humaines
        'range': f'0-{nb_offres-1}',  # Les 10 premières offres
        'minCreationDate': debut_mois,
        'maxCreationDate': fin_mois
    }

    response = requests.get(url_offres, headers=headers, params=params)

    try:
        return response.json()
    except:
        raise Exception(f"Erreur lors de la requête : {response.json()}")



token = obtenir_token(client_id, client_secret)
offres_Data_novembre = rechercher_offres(token, 'Data', 11, 2023)  # Exemple pour novembre 2023
# json to dataframe
# jsoon to dataframe
df = pd.DataFrame.from_dict(offres_Data_novembre['resultats'])


In [61]:
from datetime import datetime, timedelta

def get_access_token():
    url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=%2Fpartenaire"
    client_id = "PAR_projetdata_20dc94f3568457527fe4edda39e827c960f36708a61e6bc3b9e0bdd5c798091d"
    client_secret = "58278acf29c5d865bdc657b658ec037342ba4c889459d16c09045b5b5437986f"
    scope = "api_offresdemploiv2 application_PAR_projetdata_20dc94f3568457527fe4edda39e827c960f36708a61e6bc3b9e0bdd5c798091d o2dsoffre"
    grant_type = "client_credentials"

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
    }

    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "scope": scope,
        "grant_type": grant_type,
    }

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

def get_job_offers(access_token, min_creation_date, max_creation_date):
    url = "https://api.emploi-store.fr/partenaire/offresdemploi/v2/offres/search"
    params = {
        "minCreationDate": min_creation_date,
        "maxCreationDate": max_creation_date
    }
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }

    response = requests.get(url, headers=headers, params=params)
    try :
        return response.json()
    except :
        pass

# Création du DataFrame vide
df = pd.DataFrame()

current_date = datetime.now()

# Boucle pour les 100 derniers jours
# Boucle pour les jours spécifiés (ici, 100 à 200 derniers jours)
for day in range(901, 2001):
    # Calculer la date il y a 'day' jours
    target_date = current_date - timedelta(days=day)
    
    # Formatter la date dans le format attendu
    min_date = target_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    max_date = (target_date + timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%SZ")

    # Obtenez le token d'accès
    access_token = get_access_token()

    # Récupérez les offres d'emploi
    job_offers = get_job_offers(access_token, min_date, max_date)

    # Vérifiez si job_offers est non nul et contient la clé "resultats"
    if job_offers and "resultats" in job_offers:
        # Ajoutez les offres d'emploi au DataFrame
        df = pd.concat([df, pd.json_normalize(job_offers["resultats"])])
    else:
        print(f"Aucune donnée trouvée ou erreur pour la date : {min_date}")

# Réinitialisez l'index du DataFrame
df.reset_index(drop=True, inplace=True)

# Affichez le DataFrame final
print(df)

Aucune donnée trouvée ou erreur pour la date : 2021-07-24T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-23T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-22T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-21T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-20T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-19T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-18T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-17T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-16T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-15T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-14T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-13T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-12T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2021-07-11T11:22:37Z
Aucune donnée trouvée ou erreur pour la date : 2

In [62]:
df

,id,intitule,description,dateCreation,dateActualisation,romeCode,romeLibelle,appellationlibelle,typeContrat,typeContratLibelle,...,lieuTravail.codePostal,lieuTravail.commune,entreprise.nom,entreprise.description,entreprise.entrepriseAdaptee,origineOffre.origine,origineOffre.urlOrigine,origineOffre.partenaires,qualificationCode,qualificationLibelle
0,7391501,Poissonnier (H/F),"Mettre en scène ,animer l'étal .theatraliser g...",2021-07-03T20:35:43.000Z,2024-01-05T02:57:29.000Z,D1105,Poissonnerie,Poissonnier / Poissonnière,CDI,Contrat à durée indéterminée,...,95500,95277,Leclerc Gonesse,"Hypermarché 6000 m&sup2;, au sein d'un centre ...",False,2,https://candidat.pole-emploi.fr/offres/recherc...,"[{'nom': 'DISTRIJOB', 'url': 'https://www.dist...",NaN,NaN
1,7165513,Comptable (H/F),"Vous assistez le responsable administratif, co...",2021-06-30T15:02:46.000Z,2024-01-05T02:57:29.000Z,M1203,Comptabilité,Comptable,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,Hypermarché à 1H15 de Béziers cherche son\ncom...,False,2,https://candidat.pole-emploi.fr/offres/recherc...,"[{'nom': 'DISTRIJOB', 'url': 'https://www.dist...",NaN,NaN
2,7165043,Responsable bazar (H/F),"Directement sous l'autorité du Directeur, vous...",2021-06-30T14:59:58.000Z,2024-01-05T02:57:48.000Z,D1301,Management de magasin de détail,Responsable de boutique,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,Hypermarché leader sur sa zone recherche son \...,False,2,https://candidat.pole-emploi.fr/offres/recherc...,"[{'nom': 'DISTRIJOB', 'url': 'https://www.dist...",NaN,NaN
3,6947225,Chef de rayon boulangerie viennoiserie (H/F),Sous l'autorité du directeur et en collaborati...,2021-06-26T17:58:32.000Z,2024-01-05T02:57:39.000Z,D1502,Management/gestion de rayon produits alimentaires,Chef de rayon boulangerie,CDI,Contrat à durée indéterminée,...,NaN,NaN,Palmer International,Hypermarché indépendant de 8 000 m&sup2;\nChef...,False,2,https://candidat.pole-emploi.fr/offres/recherc...,"[{'nom': 'DISTRIJOB', 'url': 'https://www.dist...",NaN,NaN
4,5652314,Responsable magasin (H/F),Vous avez en charge l'ensemble de la gestion c...,2021-06-04T20:41:00.000Z,2024-01-09T20:25:04.000Z,D1301,Management de magasin de détail,Responsable de boutique,CDI,Contrat à durée indéterminée,...,25300,25204,Est Recrutement,"Supermarché frontalier, dynamique et agréable.",False,2,https://candidat.pole-emploi.fr/offres/recherc...,"[{'nom': 'DISTRIJOB', 'url': 'https://www.dist...",NaN,NaN
5,5551982,Manager boulangerie pâtisserie (H/F),Vous supervisez l'ensemble de la fabrication c...,2021-06-03T12:14:20.000Z,2024-01-06T21:15:10.000Z,D1502,Management/gestion de rayon produits alimentaires,Chef de rayon boulangerie/pâtisserie,CDI,Contrat à durée indéterminée,...,NaN,NaN,Est Recrutement,Hypermarché indépendant. Management dynamique ...,False,2,https://candidat.pole-emploi.fr/offres/recherc...,"[{'nom': 'DISTRIJOB', 'url': 'https://www.dist...",NaN,NaN
6,5382986,Boucher étalier (H/F),Votre expérience professionnelle vous a amené ...,2021-05-30T06:08:27.000Z,2024-01-09T20:13:23.000Z,D1101,Boucherie,Boucher / Bouchère,CDI,Contrat à durée indéterminée,...,67700,67437,Cabinet Michel Gérard,Cabinet Michel GERARD référence (Grand Est) de...,False,2,https://candidat.pole-emploi.fr/offres/recherc...,"[{'nom': 'DISTRIJOB', 'url': 'https://www.dist...",NaN,NaN
7,5276022,Menuisier bois (H/F),Dans le cadre de notre développement nous rech...,2021-05-27T16:46:10.000Z,2023-11-16T05:48:28.000Z,H2206,Réalisation de menuiserie bois et tonnellerie,Menuisier / Menuisière,CDI,Contrat à durée indéterminée,...,76700,76305,NaN,"Créée en 1931, GNC Agencement est une entrepri...",False,2,https://candidat.pole-emploi.fr/offres/recherc...,"[{'nom': 'FFB Normandie', 'url': 'https://empl...",3,"Ouvrier qualifié (P1,P2)"
8,4923003,Chef de rayon poissonnerie (H/F),Sous l'autorité du chef de département PF et e...,2021-05-20T13:07:13.000Z,2023-12-23T18:04:57.000Z,D1502,Management/gestion de rayon produits alimentaires,Chef de rayon poissonneri

In [59]:
df.columns

Index(['id', 'intitule', 'description', 'dateCreation', 'dateActualisation',
       'romeCode', 'romeLibelle', 'appellationlibelle', 'typeContrat',
       'typeContratLibelle', 'natureContrat', 'experienceExige',
       'experienceLibelle', 'alternance', 'nombrePostes', 'accessibleTH',
       'lieuTravail.libelle', 'lieuTravail.latitude', 'lieuTravail.longitude',
       'lieuTravail.codePostal', 'lieuTravail.commune', 'entreprise.nom',
       'entreprise.description', 'entreprise.entrepriseAdaptee',
       'origineOffre.origine', 'origineOffre.urlOrigine',
       'origineOffre.partenaires', 'qualificationCode',
       'qualificationLibelle'],
      dtype='object')

In [63]:
# save to csv in ../data

df.to_csv('../data/offres_900_2000.csv', index=False)


In [70]:
import pandas as pd
import os

# Define the path to the directory containing your CSV files
directory_path = "../data"

# List all CSV files in the directory
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

# Sort the list of csv files to maintain the order
csv_files.sort()

# Create an empty DataFrame to concatenate your CSV data
concatenated_df = pd.DataFrame()

# Loop over the list of csv files
for file in csv_files:
    # Construct the full path to the csv file
    file_path = os.path.join(directory_path, file)
    
    # Read the csv file and append it to the concatenated DataFrame
    df = pd.read_csv(file_path)
    concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)

# Reset the index of the concatenated DataFrame
concatenated_df.reset_index(drop=True, inplace=True)

# Optionally, save the concatenated data to a new CSV file
concatenated_df.to_csv('../data/concatenated_offers.csv', index=False)

# Display the concatenated DataFrame
print(concatenated_df)


            id                                           intitule  \
0      162RQFM  Technicien / Technicienne en mesures physiques...   
1      162RQCS  Agent / Agent (e) d'entretien des bâtiments et...   
2      162RQBK  Aide-soignant / Aide-soignante (H/F) ou AES de...   
3      162RPNJ                         Ambulancier Diplômé  (H/F)   
4      162RPHL             Dessinateur / Dessinatrice du bâtiment   
...        ...                                                ...   
26409  3541232                           Boucher / Bouchère (H/F)   
26410  3541005                       Boulanger / Boulangère (H/F)   
26411  3540910                                Chef d'équipe (H/F)   
26412  3540839             Serveur / Serveuse de restaurant (H/F)   
26413  3540732                               Réceptionnaire (H/F)   

                                             description  \
0      Dans le cadre du développement de notre activi...   
1      Recrute en urgence pour une prise de poste i